# FIT5196 Assessment 1
## Task 1
#### Student Name: Anirban Roy Chowdhury
#### Student ID: 30539676

Date: 13/09/20

Version: 3.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* re (for regular expression) 
* os (for navigating to dir of the file.)
* Langid (For language classification)


# 1. Introduction
The first part of the assignment, touches upon the first steps of analyzing textual data, i.e extracting data from semi-structured text files.
We are given about 2500 text files, containing responses from the twitter API and the data is in JSON format. 

Our tasks in as follows:
* Read & Process all the data.
* Crawl through the data to find the required information.
* Output the desired infomration in an XML format file, with specific constraints as given in the assignment.

More details for each task will be given in the following sections.

## Importing Libraries.

In [1]:
import os,re
from langid.langid import LanguageIdentifier, model

# 2. Methodology

## 2.1.Examining and Reading Data

By examining a the text file, we can see that it is a JSON file, received from the twitter API. We can confirm this guess by passing the given file through an online JSON viewer.

Our first task is to read all the data given to us for processing. Using the OS lib, we are able to walk to the required directory. Our files our "utf-8" encoded, each entry within the "new_list" variable contains all the raw text within the one of the files. 

In [2]:
new_list = []
#OS walk to the required directory, we read the text from each individual files and append the entire tezt into the list
for root, dirs, files in os.walk("D:\\2.Uni Work\\FIT5196\\Assignment 1\\30539676\\"):
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='utf8') as f:
            text = f.read()
            #new_list[re.search(r'\d{4}-\d{2}-\d{2}',str(file)).group()] = text
            new_list.append(text)

<p>Just by looking at one of the enteries within "new_list" we can see that the data within the text files are JSON responses.</p>
<p>
Since a JSON is nothing but a complicated python dictionary, we can use the in-built "eval()" function to return a python dict version of the JSON data stored within the text files.</p>
<i>from python doc</i>
<p>eval() - The expression argument is parsed and evaluated as a Python expression</p>
<p>
However here we face a problem, our data has lots of keys, whose value are "true" and "false", since in python these boolean values are special words and are defined as "True" & "False". if we directly use eval() to get our python dictionary, it will fail.
    
One thing to note is that within the twitter data, we have a "data" and "error" list. The data list contains all tweets, while the error list contains appropriate error logs from the reponses of the twitter API. our assignmentment only focuses on the data part of the responses, as such the error details are irrelevant to us in terms of this assignment.
<p>
We will be using regex to solve this issue, by replacing all true and false with their capitalized versions, we will be able to use eval() to get our desired output,saving us time by not using regex to manually crawl through the data.
</p>

## 2.2. Parsing & Extracting the Required Data

We will intialize all the required classifiers and dictionaries for further computation here.

* <b>identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)</b> - This is a Langid object which classifies the language of the tweet, we have normalized the probability so that it comes within the range of 0 to 1.
* <b>d = {'true': 'True', 'false': 'False'} </b> - This is a dictionary containing all the words that are to be replaced as keys, with the value being teh words they are to be replaced with.
* <b>pattern = re.compile(r"(true|false)")</b> - Finally we compile a regex object containing the pattern that we want to find. In our case its <u>"true|false"</u>

In [3]:
#initializing the required empty dictionaries and lists
wrangled_data = {}
dict_sorted_acc_date = {}
#Initializing our classified
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
#initializing dicts, containing words to be changed for manipulation. The key is the word to search for and the value is the word to replace it with
d = {'true': 'True', 'false': 'False'}
#regex to replace the above defined dict
pattern = re.compile(r"(true|false)")

<p>
Following are the steps taken in the bellow cell to extract the required data as per the assignment.

Steps:

1. Iterate over each entry within "new_list", which is the raw text data within one file.
2. Replace "true" and "false" with "True" and "False".
3. Using **eval()** get the corresponsing python dict from the text data.
4. Iterate over the returned dict from eval().
5. When the key matches "data", for every item within the "data" list of that file.
6. Encode the text to utf-16 and then decode it back to utf-8, since according to the assignment specification emoji's are to be displayed as it.
7. Since we have been given **utf-8** files, if we dont encode it into **utf-16** and then back, the emoji will be represented as the corresponsing combination of utf-8 charecters to depict the emoji. According to our assignment specification we want the emoji to be represented by the associated image.
8. Since **Langid** cannot work in parallel, each text has to be individually passed through our function to get which language it belongs to. Filter out any non-english language.
8. All english language tweets are stored in a dictionary where the "tweet id" is the key. If there are multiple tweets with same id, the most recent one is kept within the dict.
9. We also use regex to get the date from the tweet, while storing it in "wrangled_data" dict.

</p>

In [4]:
#Iterate through each item in the created list
for val in new_list:
    #Replace our defined words for the raw_text
    raw_text = pattern.sub(lambda m: d.get(m.group(1), m.group(1)), val)
    #Since ouur input is just a JSON file. We can use eval to get our data into dict format.
    evaluated_dict = eval(raw_text)
    #Iterate over the dict 
    for key, value in evaluated_dict .items():
        #On condition match
        if key == 'data':
            #Iterate over the list of values containing all the tweets
            for item in value:
                #each tweet text id encoded into utf16 surrogate to handle twitter emojis, since in utf16 2 utf8 charecter combine to create a emoji
                utf8_utf16_encoded_string = item['text'].encode('utf-16', 'surrogatepass').decode('utf-16')
                #If the tweet is classified as an english tweet
                if identifier.classify(utf8_utf16_encoded_string)[0] == 'en':
                    #Add the tweet into a dict along with its created_at date in its required format using regex
                    wrangled_data[item['id']] = [utf8_utf16_encoded_string, re.search(r'\d{4}-\d{2}-\d{2}',item['created_at']).group()]

<p>
The wrangled_data dict contains a list of value, where the value on first index is the twitter text and the value on second index is the date.
</p>

<p> Since we have to write our data to an xml file, we will reformat our dict "wrangled_dict" into another dict, which is has its <u>key</u> as the Date and is sorted ascendingly accodring to it. </p>
<p>Now in this new dict each Key i.e date has a list of value, within this list there are multiple lists each containing the ID & text of a single tweet.</p>

In [5]:
#We reformat our wrangled_data dictionary into a more manageble format for printing to an .xml file, by grouping all the tweets accoriding to their created_at date
for key, value in sorted(wrangled_data.items()):
    dict_sorted_acc_date.setdefault(value[1], []).append([key,value[0]]) 

## 2.3. Writing XML files.

By iterating over the "dict_sorted_acc" we can output our desired xml files, encoded in utf-8.

In [6]:
#Write to the desired outputfile according to the given schema
with open('30539676.xml','w+',encoding='utf-8') as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write('<data>\n')
    for key, val in dict_sorted_acc_date.items():
        f.write('<tweets date="%s">\n' % (key))
        for item in val:
            f.write('<tweet id="%s">%s</tweet>\n' % (item[0],item[1]))
        f.write('</tweets>\n')
    f.write('</data>')

# 3. Conclusion

<p>
Following tasks have been completed:
    
    1. Reading all .txt files given.
    2. Replacing "true" & "false" with True and False.
    3. Using eval() to get a python dict to work upon.
    4. Filtering the required non-english tweets using Langid.
    5. Converting utf-8 encoded charecter to utf-16 then back, to preserve emojis.
    6. Creating a dict containing the relevant information
    7. Generating the required XML file.
    
</p>